In [48]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [49]:
df = pd.read_excel("Train.xlsx")

In [50]:
df

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302
...,...,...,...,...,...,...,...,...,...,...,...
10678,Air Asia,9/04/2019,Kolkata,Banglore,CCU → BLR,19:55,22:25,2h 30m,non-stop,No info,4107
10679,Air India,27/04/2019,Kolkata,Banglore,CCU → BLR,20:45,23:20,2h 35m,non-stop,No info,4145
10680,Jet Airways,27/04/2019,Banglore,Delhi,BLR → DEL,08:20,11:20,3h,non-stop,No info,7229
10681,Vistara,01/03/2019,Banglore,New Delhi,BLR → DEL,11:30,14:10,2h 40m,non-stop,No info,12648


In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


In [52]:
## Prepare proposal document and final presentation.

In [53]:
# Question:
## 5w and h ## 
# 1. Which airline is most preffered airline.
# 2. Mean/Median price for which airline is highest and lowest.
# 3. For each flight which is the most common source and destination airport.
# 4. How many flights are going from each source airports and destination airport.
# 5. Which airline is having most non-stop flight.
# 6. Which day of week we have maximum and minimum flights?
# 7. During what time day we have more flights?
# 8. What is the average price for flights on each day of week?
#      a. Different hours of day [morning/afternoon/evening/night] what is the average price?
# 9. How price and duration of flight is varring?
# 10. For which source and destiantion you have maximum and minimum duration?

# Sanity Check

In [54]:
# check for duplicate data
# Date of Jorney is object need to be converted into Datetime.
# Dep_Time is fine.
# Arrival Time is incorrectly placed so need to fixed.
# Duration is available in HH:MM format need to be either in minutes or hours. checked
# Total_Stops should be numerical but right it is object.
# Check if Additional_Info column provides any meaning information or not.

In [55]:
## Drop Duplicates
df.drop_duplicates(inplace=True)

In [56]:
def time_minute(duration):
    dur = duration.split()
    min = 0
    for x in dur:
        if x[-1] == 'h':
            min = min + int(x[:-1])*60

        if x[-1] == 'm':
            min = min + int(int(x[:-1])) 
    return min

def convert_duration_to_minutes(time):
    '''
    This function converts duration in h m to minutes:
    input : hh:mm ,hh, mm
    return:
        min
    '''
    if len(time.split(' ')) >1 :
        hh,mm = time.split(' ')
        hh,mm = int(hh[:-1]),int(mm[:-1])
        duration = hh*60+mm
    else:
        if 'h' in time:
            duration = int(time[:-1])*60
        else:
            duration= int(time[:-1])
            
    return duration

In [57]:
# df['DepartureDateTime'] = df['Date_of_Journey'] + " "+ df['Dep_Time']
# df['DepartureDateTime'] = pd.to_datetime(df['DepartureDateTime'],infer_datetime_format=True)
# df['Duration_min'] = df['Duration'].apply(lambda x: convert_duration_to_minutes(x))
# df['Duration_timedelta'] = pd.to_timedelta(df['Duration_min'], unit='m')
# df["ArrivalDateTime"] = df['DepartureDateTime'] + df['Duration_timedelta']

def create_arrival_time(df):
    '''
    This Function preprocess date_of_journey and duration to create departure and arrival date time.
    '''
    df['DepartureDateTime'] = df['Date_of_Journey'] + " "+ df['Dep_Time']
    df['DepartureDateTime'] = pd.to_datetime(df['DepartureDateTime'],infer_datetime_format=True)
    df['Duration_min'] = df['Duration'].apply(lambda x: convert_duration_to_minutes(x))
    df['Duration_timedelta'] = pd.to_timedelta(df['Duration_min'], unit='m')
    df["ArrivalDateTime"] = df['DepartureDateTime'] + df['Duration_timedelta']
    return df

create_arrival_time(df)

C:\Users\user\AppData\Local\Temp\ipykernel_2348\2267467950.py:12: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['DepartureDateTime'] = pd.to_datetime(df['DepartureDateTime'],infer_datetime_format=True)
C:\Users\user\AppData\Local\Temp\ipykernel_2348\2267467950.py:12: UserWarning: Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['DepartureDateTime'] = pd.to_datetime(df['DepartureDateTime'],infer_datetime_format=True)


,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,DepartureDateTime,Duration_min,Duration_timedelta,ArrivalDateTime
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897,2019-03-24 22:20:00,170,0 days 02:50:00,2019-03-25 01:10:00
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662,2019-05-01 05:50:00,445,0 days 07:25:00,2019-05-01 13:15:00
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882,2019-06-09 09:25:00,1140,0 days 19:00:00,2019-06-10 04:25:00
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218,2019-05-12 18:05:00,325,0 days 05:25:00,2019-05-12 23:30:00
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302,2019-03-01 16:50:00,285,0 days 04:45:00,2019-03-01 21:35:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10678,Air Asia,9/04/2019,Kolkata,Banglore,CCU → BLR,19:55,22:25,2h 30m,non-stop,No info,4107,2019-04-09 19:55:00,150,0 days 02:30:00,2019-04-09 22:25:00
10679,Air India,27/04/2019,Kolkata,Banglore,CCU → BLR,20:45,23:20,2h 35m,non-stop,No info,4145,2019-04-27 20:45:00,155,0 days 02:35:00,2019-04-27 23:20:00
10680,Jet Airways,27/04/2019,Banglore,Delhi,BLR → DEL,08:20,11:20,3h,non-stop,No info,7229,2019-04-27 08:20:00,180,0 days 03:00:00,2019-04-27 11:20:00
10681,Vistara,01/03/2019,Banglore,New Delhi,BLR → DEL,11:30,14:10,2h 40m,non-stop,No info,12648,2019-03-01 11:30:00,160,0 days 02:40:00,2019-03-01 14:10:00


In [58]:
df.drop(columns=['Date_of_Journey','Dep_Time','Arrival_Time','Duration'],axis=1,inplace=True)

In [59]:
stops_dict = {
    'non-stop':0,
    '2 stops':2,
    '1 stop':1,
    '3 stops':3,
    '4 stops':4
}
df['Total_Stops'] = df['Total_Stops'].replace(stops_dict)

In [60]:
df['Additional_Info'].value_counts()/len(df)*100

Additional_Info
No info                         78.208927
In-flight meal not included     18.407722
No check-in baggage included     3.039281
1 Long layover                   0.181592
Change airports                  0.066902
Business class                   0.038230
No Info                          0.028672
1 Short layover                  0.009557
Red-eye flight                   0.009557
2 Long layover                   0.009557
Name: count, dtype: float64

In [61]:
# Additional Info columns doesnot have any meaningful information.
df.drop(columns=['Additional_Info'],axis=1,inplace=True)

In [62]:
df.head(2)

,Airline,Source,Destination,Route,Total_Stops,Price,DepartureDateTime,Duration_min,Duration_timedelta,ArrivalDateTime
0,IndiGo,Banglore,New Delhi,BLR → DEL,0.0,3897,2019-03-24 22:20:00,170,0 days 02:50:00,2019-03-25 01:10:00
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,2.0,7662,2019-05-01 05:50:00,445,0 days 07:25:00,2019-05-01 13:15:00


#### Actions on Sanity Check

In [63]:
def convert_duration_to_minutes(time):
    '''
    This function converts duration in h m to minutes:
    input : hh:mm ,hh, mm
    return:
        min
    '''
    if len(time.split(' ')) >1 :
        hh,mm = time.split(' ')
        hh,mm = int(hh[:-1]),int(mm[:-1])
        duration = hh*60+mm
    else:
        if 'h' in time:
            duration = int(time[:-1])*60
        else:
            duration= int(time[:-1])
            
    return duration


def create_preprocess_date_time(df):
    '''
    This Function preprocess date_of_journey and duration to create departure and arrival date time.
    '''
    df['DepartureDateTime'] = df['Date_of_Journey'] + " "+ df['Dep_Time']
    df['DepartureDateTime'] = pd.to_datetime(df['DepartureDateTime'],infer_datetime_format=True)
    df['Duration_min'] = df['Duration'].apply(lambda x: convert_duration_to_minutes(x))
    df['Duration_timedelta'] = pd.to_timedelta(df['Duration_min'], unit='m')
    df["ArrivalDateTime"] = df['DepartureDateTime'] + df['Duration_timedelta']
    return df

stops_dict = {
    'non-stop':0,
    '2 stops':2,
    '1 stop':1,
    '3 stops':3,
    '4 stops':4
}

## Running the process of prediction is also referred as inference:
def sanity_check(df,train=True):
    '''
    This function performs sanity check on the airline data.
    inputs:
        df: dataframe that we need to perform sanity check
        train: This is used for process of training and inference.
            train is having default value of True and can be set as False if we are running inference.
            
        ## process for training    
        sanity_check(df)
        # process of prediction
        sanity_check(df,train=False)
    returns:
        df
    '''
    if train:
        df.drop_duplicates(inplace=True)
    
    create_preprocess_date_time(df)
    df['Total_Stops'] = df['Total_Stops'].replace(stops_dict)
    df.drop(columns=['Date_of_Journey','Dep_Time','Arrival_Time','Duration','Additional_Info'],axis=1,inplace=True)
    
    return df
    
## process for training    
sanity_check(df)
# process of prediction
sanity_check(df,train=False)

KeyError: 'Date_of_Journey'

# Missing value Handling

In [ ]:
# dropping Nan value as its a single observation.
df.dropna(inplace=True)

#### Actions on missing value

In [ ]:
def handle_missing_value(df,train=True):
    """
    This function helps to handle missing value.
    Since for Airline data there is just one missing value we can choose to drop missing value.
    inputs:
         df: dataframe which requires imputation.
         
    returns:
        df
    
    """
    df.dropna(inplace=True)
    return df

handle_missing_value(df)

# Handling Categorical Values

In [64]:
df['Destination'] = df['Destination'].replace({'New Delhi':'Delhi'})

In [65]:
# No order in columns
categorical_cols = ['Airline', 'Source', 'Destination', 'Route']
for col in categorical_cols:
    print("Column :",col," Unique values :",df[col].unique()," and No. of uniques : ",df[col].nunique())
    print("-"*90)

Column : Airline  Unique values : ['IndiGo' 'Air India' 'Jet Airways' 'SpiceJet' 'Multiple carriers' 'GoAir'
 'Vistara' 'Air Asia' 'Vistara Premium economy' 'Jet Airways Business'
 'Multiple carriers Premium economy' 'Trujet']  and No. of uniques :  12
------------------------------------------------------------------------------------------
Column : Source  Unique values : ['Banglore' 'Kolkata' 'Delhi' 'Chennai' 'Mumbai']  and No. of uniques :  5
------------------------------------------------------------------------------------------
Column : Destination  Unique values : ['Delhi' 'Banglore' 'Cochin' 'Kolkata' 'Hyderabad']  and No. of uniques :  5
------------------------------------------------------------------------------------------
Column : Route  Unique values : ['BLR → DEL' 'CCU → IXR → BBI → BLR' 'DEL → LKO → BOM → COK'
 'CCU → NAG → BLR' 'BLR → NAG → DEL' 'CCU → BLR' 'BLR → BOM → DEL'
 'DEL → BOM → COK' 'DEL → BLR → COK' 'MAA → CCU' 'CCU → BOM → BLR'
 'DEL → AMD → BOM → COK'

In [66]:
col = 'Airline'

In [67]:
df.groupby(col).size()

Airline
Air Asia                              318
Air India                            1695
GoAir                                 194
IndiGo                               2043
Jet Airways                          3700
Jet Airways Business                    6
Multiple carriers                    1196
Multiple carriers Premium economy      13
SpiceJet                              815
Trujet                                  1
Vistara                               477
Vistara Premium economy                 3
dtype: int64

In [68]:
freq_value = df.groupby(col).size()/len(df)

In [69]:
freq_value

Airline
Air Asia                             0.030399
Air India                            0.162030
GoAir                                0.018545
IndiGo                               0.195297
Jet Airways                          0.353695
Jet Airways Business                 0.000574
Multiple carriers                    0.114329
Multiple carriers Premium economy    0.001243
SpiceJet                             0.077908
Trujet                               0.000096
Vistara                              0.045598
Vistara Premium economy              0.000287
dtype: float64

In [70]:
freq_dict = freq_value.to_dict()

In [71]:
freq_dict

{'Air Asia': 0.030398623458560366,
 'Air India': 0.16203039862345855,
 'GoAir': 0.018545072172832425,
 'IndiGo': 0.19529681674792085,
 'Jet Airways': 0.35369467546123695,
 'Jet Airways Business': 0.0005735589331803843,
 'Multiple carriers': 0.1143294140139566,
 'Multiple carriers Premium economy': 0.0012427110218908325,
 'SpiceJet': 0.0779084217570022,
 'Trujet': 9.559315553006405e-05,
 'Vistara': 0.04559793518784055,
 'Vistara Premium economy': 0.00028677946659019213}

In [72]:
df["Freq_encoded_"+col] = df[col].replace(freq_dict)

In [73]:
df.head(1)

,Airline,Source,Destination,Route,Total_Stops,Price,DepartureDateTime,Duration_min,Duration_timedelta,ArrivalDateTime,Freq_encoded_Airline
0,IndiGo,Banglore,Delhi,BLR → DEL,0.0,3897,2019-03-24 22:20:00,170,0 days 02:50:00,2019-03-25 01:10:00,0.195297


In [74]:
def frequency_encoder(df,col):
    """
    This function encodes a categorical column based on the frequency of their occurence.
    input:
        df : Input DataFrame in which encoding has to be created 
        col : Column name which has to be encoded
    return: 
          frequency encoded dictionary for columns
    """
    freq_value = df.groupby(col).size()/len(df)
    freq_dict = freq_value.to_dict()
    df["Freq_encoded_"+col] = df[col].replace(freq_dict)
    return freq_dict

In [75]:
col = 'Airline'
target_col = 'Price'

In [76]:
mean_value = df.groupby(col)[target_col].mean()

In [77]:
mean_value

Airline
Air Asia                              5593.745283
Air India                             9555.382891
GoAir                                 5861.056701
IndiGo                                5668.469897
Jet Airways                          11599.021081
Jet Airways Business                 58358.666667
Multiple carriers                    10902.678094
Multiple carriers Premium economy    11418.846154
SpiceJet                              4335.841718
Trujet                                4140.000000
Vistara                               7801.761006
Vistara Premium economy               8962.333333
Name: Price, dtype: float64

In [78]:
mean_dict = mean_value.to_dict()

In [79]:
mean_dict

{'Air Asia': 5593.745283018868,
 'Air India': 9555.382890855457,
 'GoAir': 5861.056701030928,
 'IndiGo': 5668.469897209985,
 'Jet Airways': 11599.02108108108,
 'Jet Airways Business': 58358.666666666664,
 'Multiple carriers': 10902.678093645485,
 'Multiple carriers Premium economy': 11418.846153846154,
 'SpiceJet': 4335.841717791411,
 'Trujet': 4140.0,
 'Vistara': 7801.761006289308,
 'Vistara Premium economy': 8962.333333333334}

In [80]:
df["Mean_encoded_"+col] = df[col].replace(mean_dict)

In [81]:
df.head(1)

,Airline,Source,Destination,Route,Total_Stops,Price,DepartureDateTime,Duration_min,Duration_timedelta,ArrivalDateTime,Freq_encoded_Airline,Mean_encoded_Airline
0,IndiGo,Banglore,Delhi,BLR → DEL,0.0,3897,2019-03-24 22:20:00,170,0 days 02:50:00,2019-03-25 01:10:00,0.195297,5668.469897


In [82]:
def mean_encoder(df,col,target_col):
    """
    This function encodes a categorical column based on the frequency of their occurence.
    input:
        df : Input DataFrame in which encoding has to be created 
        col : Column name which has to be encoded
    return: 
          Mean encoded dict for column
    """
    mean_value = df.groupby(col)[target_col].mean()
    mean_dict = mean_value.to_dict()
    df["Mean_encoded_"+col] = df[col].replace(mean_dict)
    return mean_dict

In [83]:
## Label encoder for function and later usages:

## Applying Label Encoder to Categorical columns as Hit and Trail.
from sklearn.preprocessing import LabelEncoder

def label_encoder(df,col):
    """
    This function encodes a categorical column based on the basis of their order label.
    input:
        df : Input DataFrame in which encoding has to be created 
        col : Column name which has to be encoded
    return: 
          label encoded dict for column
    """
    le = LabelEncoder()
    le.fit(df[col])
    label_dict = dict(zip((le.classes_),le.transform(le.classes_)))
    df["Label_encoded_"+col] = df[col].replace(label_dict)
    return label_dict

In [84]:
## Create a function to handle categorical value
def handle_categorical_values(df,target):
    '''
      This function handles categorical value and create a dataframe.
      Input:
        df : Dataframe which require categorical value treatment
      returns :
         Dataframe with all categorical value handled.
    '''
    encoded_dict = {}
    # Getting all object columns
    object_columns = df.select_dtypes(object).columns

    ## generate frequency encoded categorical values
    frequency_encoded_dict ={} 
    for col in object_columns:
        freq_dict = frequency_encoder(df,col)
        frequency_encoded_dict[col] = freq_dict

    ## generate target mean encoded categorical values
    mean_encoded_dict ={} 
    for col in object_columns:
        mean_dict = mean_encoder(df,col,target)
        mean_encoded_dict[col] = mean_dict

    
    ## generate label encoded categorical values
    label_encoded_dict ={} 
    for col in object_columns:
        label_dict = label_encoder(df,col)
        label_encoded_dict[col] = label_dict
    
    encoded_dict["Frequency"] = frequency_encoded_dict
    encoded_dict["Mean"] = mean_encoded_dict
    encoded_dict["Label"] = label_encoded_dict

    return df, encoded_dict

In [85]:
df.select_dtypes(object).columns

Index(['Airline', 'Source', 'Destination', 'Route'], dtype='object')

In [86]:
target = 'Price'
df, encoded_dict = handle_categorical_values(df,target)

In [87]:
encoded_dict

{'Frequency': {'Airline': {'Air Asia': 0.030398623458560366,
   'Air India': 0.16203039862345855,
   'GoAir': 0.018545072172832425,
   'IndiGo': 0.19529681674792085,
   'Jet Airways': 0.35369467546123695,
   'Jet Airways Business': 0.0005735589331803843,
   'Multiple carriers': 0.1143294140139566,
   'Multiple carriers Premium economy': 0.0012427110218908325,
   'SpiceJet': 0.0779084217570022,
   'Trujet': 9.559315553006405e-05,
   'Vistara': 0.04559793518784055,
   'Vistara Premium economy': 0.00028677946659019213},
  'Source': {'Banglore': 0.20810629958894944,
   'Chennai': 0.0364209922569544,
   'Delhi': 0.41544785393365835,
   'Kolkata': 0.2733964248159832,
   'Mumbai': 0.06662842940445464},
  'Destination': {'Banglore': 0.2733964248159832,
   'Cochin': 0.41544785393365835,
   'Delhi': 0.20810629958894944,
   'Hyderabad': 0.06662842940445464,
   'Kolkata': 0.0364209922569544},
  'Route': {'BLR → AMD → DEL': 0.0017206767995411529,
   'BLR → BBI → DEL': 0.00047796577765032023,
   'BL

In [88]:
df.head(2)

,Airline,Source,Destination,Route,Total_Stops,Price,DepartureDateTime,Duration_min,Duration_timedelta,ArrivalDateTime,...,Freq_encoded_Source,Freq_encoded_Destination,Freq_encoded_Route,Mean_encoded_Source,Mean_encoded_Destination,Mean_encoded_Route,Label_encoded_Airline,Label_encoded_Source,Label_encoded_Destination,Label_encoded_Route
0,IndiGo,Banglore,Delhi,BLR → DEL,0.0,3897,2019-03-24 22:20:00,170,0 days 02:50:00,2019-03-25 01:10:00,...,0.208106,0.208106,0.146640,8024.689940,8024.689940,5551.593220,3,0,2,18.0
1,Air India,Kolkata,Banglore,CCU → IXR → BBI → BLR,2.0,7662,2019-05-01 05:50:00,445,0 days 07:25:00,2019-05-01 13:15:00,...,0.273396,0.273396,0.000574,9143.083566,9143.083566,7369.166667,1,3,0,84.0


In [89]:
categorical_cols = ['Airline', 'Source', 'Destination', 'Route']
df.drop(columns=categorical_cols,inplace=True)

#### Action on categorical Values

In [90]:
def frequency_encoder(df,col):
    """
    This function encodes a categorical column based on the frequency of their occurence.
    input:
        df : Input DataFrame in which encoding has to be created 
        col : Column name which has to be encoded
    return: 
          frequency encoded dictionary for columns
    """
    freq_value = df.groupby(col).size()/len(df)
    freq_dict = freq_value.to_dict()
    df["Freq_encoded_"+col] = df[col].replace(freq_dict)
    return freq_dict


def mean_encoder(df,col,target_col):
    """
    This function encodes a categorical column based on the frequency of their occurence.
    input:
        df : Input DataFrame in which encoding has to be created 
        col : Column name which has to be encoded
    return: 
          Mean encoded dict for column
    """
    mean_value = df.groupby(col)[target_col].mean()
    mean_dict = mean_value.to_dict()
    df["Mean_encoded_"+col] = df[col].replace(mean_dict)
    return mean_dict


from sklearn.preprocessing import LabelEncoder

def label_encoder(df,col):
    """
    This function encodes a categorical column based on the basis of their order label.
    input:
        df : Input DataFrame in which encoding has to be created 
        col : Column name which has to be encoded
    return: 
          label encoded dict for column
    """
    le = LabelEncoder()
    le.fit(df[col])
    label_dict = dict(zip((le.classes_),le.transform(le.classes_)))
    df["Label_encoded_"+col] = df[col].replace(label_dict)
    return label_dict


## Create a function to handle categorical value
def handle_categorical_values(df,target):
    '''
      This function handles categorical value and create a dataframe.
      Input:
        df : Dataframe which require categorical value treatment
      returns :
         Dataframe with all categorical value handled.
    '''
    encoded_dict = {}
    # Getting all object columns
    object_columns = df.select_dtypes(object).columns

    ## generate frequency encoded categorical values
    frequency_encoded_dict ={} 
    for col in object_columns:
        freq_dict = frequency_encoder(df,col)
        frequency_encoded_dict[col] = freq_dict

    ## generate target mean encoded categorical values
    mean_encoded_dict ={} 
    for col in object_columns:
        mean_dict = mean_encoder(df,col,target)
        mean_encoded_dict[col] = mean_dict

    
    ## generate label encoded categorical values
    label_encoded_dict ={} 
    for col in object_columns:
        label_dict = label_encoder(df,col)
        label_encoded_dict[col] = label_dict
    
    encoded_dict["Frequency"] = frequency_encoded_dict
    encoded_dict["Mean"] = mean_encoded_dict
    encoded_dict["Label"] = label_encoded_dict

    return df, encoded_dict


def airline_handle_categorical_data(df,target):
    df['Destination'] = df['Destination'].replace({'New Delhi':'Delhi'})
    df, encoded_dict = handle_categorical_values(df,target)
    categorical_cols = df.select_dtypes(object).columns
    df.drop(columns=categorical_cols,inplace=True)
    return df, encoded_dict

df, encoded_dict = airline_handle_categorical_data(df,'Price')

KeyError: 'Destination'

## Correlation

In [91]:
df.corr()

,Total_Stops,Price,DepartureDateTime,Duration_min,Duration_timedelta,ArrivalDateTime,Freq_encoded_Airline,Mean_encoded_Airline,Freq_encoded_Source,Freq_encoded_Destination,Freq_encoded_Route,Mean_encoded_Source,Mean_encoded_Destination,Mean_encoded_Route,Label_encoded_Airline,Label_encoded_Source,Label_encoded_Destination,Label_encoded_Route
Total_Stops,1.000000,0.601978,0.032399,0.727930,0.727930,0.039369,0.211636,0.421194,0.571425,0.571425,-0.205132,0.559061,0.559061,0.806659,-0.194563,0.193899,-0.422555,0.441153
Price,0.601978,1.000000,-0.149351,0.501597,0.501597,-0.144545,0.371569,0.638622,0.350856,0.350856,0.039251,0.359852,0.359852,0.746261,-0.036549,0.013596,-0.260673,0.154164
DepartureDateTime,0.032399,-0.149351,1.000000,-0.003145,-0.003145,0.999954,0.029146,0.029206,0.160360,0.160360,0.105370,0.146240,0.146240,-0.061108,0.031828,0.178190,-0.146161,0.249640
Duration_min,0.727930,0.501597,-0.003145,1.000000,1.000000,0.006431,0.289261,0.433855,0.400089,0.400089,-0.110685,0.414909,0.414909,0.671820,-0.153788,0.162560,-0.383666,0.273175
Duration_timedelta,0.727930,0.501597,-0.003145,1.000000,1.000000,0.006431,0.289261,0.433855,0.400089,0.400089,-0.110685,0.414909,0.414909,0.671820,-0.153788,0.162560,-0.383666,0.273175
ArrivalDateTime,0.039369,-0.144545,0.999954,0.006431,0.006431,1.000000,0.031915,0.033360,0.164188,0.164188,0.104308,0.150211,0.150211,-0.054674,0.030355,0.179744,-0.149833,0.252252
Freq_encoded_Airline,0.211636,0.371569,0.029146,0.289261,0.289261,0.031915,1.000000,0.581829,0.037771,0.037771,0.019832,0.055925,0.055925,0.367429,-0.200531,0.055493,-0.107193,-0.066285
Mean_encoded_Airline,0.421194,0.638622,0.029206,0.433855,0.433855,0.033360,0.581829,1.000000,0.258407,0.258407,0.158803,0.257210,0.257210,0.572079,-0.057231,0.047567,-0.183374,0.071402
Freq_encoded_Source,0.571425,0.350856,0.160360,0.400089,0.400089,0.164188,0.037771,0.258407,1.000000,1.000000,0.303445,0.975000,0.975000,0.470222,-0.049791,0.067232,-0.595939,0.612529
Freq_encoded_Destination,0.571425,0.350856,0.160360,0.400089,0.400089,0.164188,0.037771,0.258407,1.000000,1.000000,0.303445,0.975000,0.975000,0.470222,-0.049791,0.067232,-0.595939,0.612529


In [92]:
from sklearn.model_selection import train_test_split

from sklearn import linear_model
from sklearn import metrics

from sklearn.preprocessing import StandardScaler

from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import SplineTransformer

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor


def regression(X_train, X_test, y_train, y_test):
    ## Baseline model
    reg = linear_model.LinearRegression()
    reg.fit(X_train,y_train)
    print(metrics.r2_score(y_train,reg.predict(X_train)), metrics.r2_score(y_test,reg.predict(X_test)))
    return reg

def knearestneighbour(X_train, X_test, y_train, y_test):
    knn_regressor = KNeighborsRegressor(n_neighbors=10)
    knn_regressor.fit(X_train, y_train)
    print(metrics.r2_score(y_train,knn_regressor.predict(X_train)),metrics.r2_score(y_test,knn_regressor.predict(X_test)))
    return knn_regressor

def decisiontree(X_train, X_test, y_train, y_test):
    tree = DecisionTreeRegressor(max_depth=10)
    tree.fit(X_train,y_train)
    print(metrics.r2_score(y_train,tree.predict(X_train)),metrics.r2_score(y_test,tree.predict(X_test)))
    return tree

def randomForest(X_train, X_test, y_train, y_test):
    random_regressor = RandomForestRegressor(n_estimators=500, min_samples_split=10,min_samples_leaf=2)
    random_regressor.fit(X_train, y_train)
    print(metrics.r2_score(y_train,random_regressor.predict(X_train)),metrics.r2_score(y_test,random_regressor.predict(X_test)))
    return random_regressor

from xgboost import XGBRegressor
def xgboost(X_train, X_test, y_train, y_test):
    xgboost_regressor = XGBRegressor()
    xgboost_regressor.fit(X_train, y_train)
    print(metrics.r2_score(y_train,xgboost_regressor.predict(X_train)),metrics.r2_score(y_test,xgboost_regressor.predict(X_test)))
    return xgboost_regressor

from catboost import  CatBoostRegressor
def catboost(X_train, X_test, y_train, y_test):
    catboost_regressor = CatBoostRegressor(verbose=False)
    catboost_regressor.fit(X_train, y_train)
    print(metrics.r2_score(y_train,catboost_regressor.predict(X_train)),metrics.r2_score(y_test,catboost_regressor.predict(X_test)))
    return catboost_regressor

from sklearn.ensemble import GradientBoostingRegressor
def gboost(X_train, X_test, y_train, y_test):
    gboost_regressor = GradientBoostingRegressor()
    gboost_regressor.fit(X_train, y_train)
    print(metrics.r2_score(y_train,gboost_regressor.predict(X_train)),metrics.r2_score(y_test,gboost_regressor.predict(X_test)))
    return gboost_regressor


def train(X,y,modelType):
    # Split your dataset into train and test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
    model = modelType(X_train, X_test, y_train, y_test)
    return model


In [93]:
df.head(2)

,Total_Stops,Price,DepartureDateTime,Duration_min,Duration_timedelta,ArrivalDateTime,Freq_encoded_Airline,Mean_encoded_Airline,Freq_encoded_Source,Freq_encoded_Destination,Freq_encoded_Route,Mean_encoded_Source,Mean_encoded_Destination,Mean_encoded_Route,Label_encoded_Airline,Label_encoded_Source,Label_encoded_Destination,Label_encoded_Route
0,0.0,3897,2019-03-24 22:20:00,170,0 days 02:50:00,2019-03-25 01:10:00,0.195297,5668.469897,0.208106,0.208106,0.146640,8024.689940,8024.689940,5551.593220,3,0,2,18.0
1,2.0,7662,2019-05-01 05:50:00,445,0 days 07:25:00,2019-05-01 13:15:00,0.162030,9555.382891,0.273396,0.273396,0.000574,9143.083566,9143.083566,7369.166667,1,3,0,84.0


In [94]:
X = df.drop(columns=['Price','DepartureDateTime','Duration_timedelta','ArrivalDateTime'])
y = df['Price']

In [95]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
reg = linear_model.LinearRegression()
reg.fit(X_train,y_train)
print(metrics.r2_score(y_train,reg.predict(X_train)), metrics.r2_score(y_test,reg.predict(X_test)))

ValueError: Input X contains NaN.
LinearRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
train(X,y,regression)

In [ ]:
train(X,y,knearestneighbour)

In [ ]:
train(X,y,decisiontree)

In [ ]:
train(X,y,randomForest)

In [ ]:
train(X,y,xgboost)

In [ ]:
train(X,y,catboost)

In [ ]:
train(X,y,gboost)

In [ ]:
## Convert all data into numerical form to obtain your baseline model.

In [96]:
from sklearn.linear_model import LinearRegression

def sklearn_vif(exogs, data):
    '''
    This function calculates variance inflation function in sklearn way. 
     It is a comparatively faster process.
    '''
    # initialize dictionaries
    vif_dict, tolerance_dict = {}, {}

    # form input data for each exogenous variable
    for exog in exogs:
        not_exog = [i for i in exogs if i != exog]
        X, y = data[not_exog], data[exog]

        # extract r-squared from the fit
        r_squared = LinearRegression().fit(X, y).score(X, y)

        # calculate VIF
        vif = 1/(1 - r_squared)
        vif_dict[exog] = vif

        # calculate tolerance
        tolerance = 1 - r_squared
        tolerance_dict[exog] = tolerance

    # return VIF DataFrame
    df_vif = pd.DataFrame({'VIF': vif_dict, 'Tolerance': tolerance_dict})

    return df_vif

df_vif= sklearn_vif(exogs=X.columns, data=X).sort_values(by='VIF',ascending=False)

ValueError: Input X contains NaN.
LinearRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
# VIF >= 10 shows high collinearity
df_vif

In [97]:
X.columns

Index(['Total_Stops', 'Duration_min', 'Freq_encoded_Airline',
       'Mean_encoded_Airline', 'Freq_encoded_Source',
       'Freq_encoded_Destination', 'Freq_encoded_Route', 'Mean_encoded_Source',
       'Mean_encoded_Destination', 'Mean_encoded_Route',
       'Label_encoded_Airline', 'Label_encoded_Source',
       'Label_encoded_Destination', 'Label_encoded_Route'],
      dtype='object')

In [98]:
vif_cols = ['Total_Stops', 'Duration_min', 'Freq_encoded_Airline',
       'Mean_encoded_Airline', 
#             'Freq_encoded_Source',
       'Freq_encoded_Destination', 'Freq_encoded_Route', 
#             'Mean_encoded_Source',
#        'Mean_encoded_Destination',
            'Mean_encoded_Route',
       'Label_encoded_Airline', 'Label_encoded_Source',
       'Label_encoded_Destination', 'Label_encoded_Route']
X[vif_cols]
df_vif= sklearn_vif(exogs=vif_cols, data=X[vif_cols]).sort_values(by='VIF',ascending=False)
df_vif

ValueError: Input X contains NaN.
LinearRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
train(X[vif_cols],y,regression)
train(X[vif_cols],y,knearestneighbour)
train(X[vif_cols],y,decisiontree)
train(X[vif_cols],y,randomForest)
train(X[vif_cols],y,xgboost)
train(X[vif_cols],y,gboost)
train(X[vif_cols],y,catboost)

## Scaling and Outlier treatment

In [99]:
X = X[vif_cols]

In [100]:
from sklearn.preprocessing import RobustScaler

X_transform = RobustScaler().fit_transform(X)
X_transformed = pd.DataFrame(X_transform,columns=X.columns)

train(X_transformed,y,regression)
train(X_transformed,y,knearestneighbour)
train(X_transformed,y,decisiontree)
train(X_transformed,y,randomForest)
train(X_transformed,y,xgboost)
train(X_transformed,y,gboost)
train(X_transformed,y,catboost)

ValueError: Input X contains NaN.
LinearRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
df_transformed =X_transformed.copy(deep=True)
df_transformed['Price'] = y
df_transformed.corr()

In [ ]:
from sklearn.preprocessing import PowerTransformer

X_transform = PowerTransformer().fit_transform(X)
X_transformed = pd.DataFrame(X_transform,columns=X.columns)

train(X_transformed,y,regression)
train(X_transformed,y,knearestneighbour)
train(X_transformed,y,decisiontree)
train(X_transformed,y,randomForest)
train(X_transformed,y,xgboost)
train(X_transformed,y,gboost)
train(X_transformed,y,catboost)

In [101]:
df_transformed =X_transformed.copy(deep=True)
df_transformed['Price'] = y
df_transformed.corr()

,Total_Stops,Duration_min,Freq_encoded_Airline,Mean_encoded_Airline,Freq_encoded_Destination,Freq_encoded_Route,Mean_encoded_Route,Label_encoded_Airline,Label_encoded_Source,Label_encoded_Destination,Label_encoded_Route,Price
Total_Stops,1.000000,0.727930,0.211636,0.421194,0.571425,-0.205132,0.806659,-0.194563,0.193899,-0.422555,0.441153,0.020604
Duration_min,0.727930,1.000000,0.289261,0.433855,0.400089,-0.110685,0.671820,-0.153788,0.162560,-0.383666,0.273175,0.010392
Freq_encoded_Airline,0.211636,0.289261,1.000000,0.581829,0.037771,0.019832,0.367429,-0.200531,0.055493,-0.107193,-0.066285,0.009577
Mean_encoded_Airline,0.421194,0.433855,0.581829,1.000000,0.258407,0.158803,0.572079,-0.057231,0.047567,-0.183374,0.071402,0.031378
Freq_encoded_Destination,0.571425,0.400089,0.037771,0.258407,1.000000,0.303445,0.470222,-0.049791,0.067232,-0.595939,0.612529,0.027984
Freq_encoded_Route,-0.205132,-0.110685,0.019832,0.158803,0.303445,1.000000,0.052597,0.194110,-0.195724,-0.001637,0.056397,0.030701
Mean_encoded_Route,0.806659,0.671820,0.367429,0.572079,0.470222,0.052597,1.000000,-0.058117,0.018220,-0.349316,0.206667,0.039173
Label_encoded_Airline,-0.194563,-0.153788,-0.200531,-0.057231,-0.049791,0.194110,-0.058117,1.000000,-0.012048,0.068618,0.029601,0.011530
Label_encoded_Source,0.193899,0.162560,0.055493,0.047567,0.067232,-0.195724,0.018220,-0.012048,1.000000,-0.432494,0.403561,-0.005722
Label_encoded_Destination,-0.422555,-0.383666,-0.107193,-0.183374,-0.595939,-0.001637,-0.349316,0.068618,-0.432494,1.000000,-0.229317,-0.011623


In [102]:
from sklearn.preprocessing import FunctionTransformer

X_transform = FunctionTransformer(np.log1p).fit_transform(X)
X_transformed = pd.DataFrame(X_transform,columns=X.columns)


train(X_transformed,y,regression)
train(X_transformed,y,knearestneighbour)
train(X_transformed,y,decisiontree)
train(X_transformed,y,randomForest)
train(X_transformed,y,xgboost)
train(X_transformed,y,gboost)
train(X_transformed,y,catboost)

ValueError: Input X contains NaN.
LinearRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
df_transformed =X_transformed.copy(deep=True)
df_transformed['Price'] = y
df_transformed.corr()

In [103]:
from sklearn.preprocessing import FunctionTransformer

def cube(X): return X**(0.333)

X_transform = FunctionTransformer(cube).fit_transform(X)
X_transformed = pd.DataFrame(X_transform,columns=X.columns)


train(X_transformed,y,regression)
train(X_transformed,y,knearestneighbour)
train(X_transformed,y,decisiontree)
train(X_transformed,y,randomForest)
train(X_transformed,y,xgboost)
train(X_transformed,y,gboost)
train(X_transformed,y,catboost)

ValueError: Input X contains NaN.
LinearRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
df_transformed =X_transformed.copy(deep=True)
df_transformed['Price'] = y
df_transformed.corr()

In [104]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(degree=2, include_bias=False,interaction_only=False)

X_transform = poly.fit_transform(X)
X_transformed = pd.DataFrame(X_transform,columns=poly.get_feature_names(X.columns))

train(X_transformed,y,regression)
train(X_transformed,y,knearestneighbour)
train(X_transformed,y,decisiontree)
train(X_transformed,y,randomForest)
train(X_transformed,y,xgboost)
train(X_transformed,y,gboost)
train(X_transformed,y,catboost)

ValueError: Input X contains NaN.
PolynomialFeatures does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
df_transformed =X_transformed.copy(deep=True)
df_transformed['Price'] = y
df_transformed.corr()

## Adding Basic Time based Features

## I

In [ ]:
def time_of_day(hr):
    '''
    This function gives the time of day based on logic:
        # 3-8 early_morning or 1
        # 8-12 morning or 2
        # 12-16 afternoon or 3
        # 16-20 evening or 4
        # 20-00 night or 5
        # 00-3 late_night or 6
        # invalid or 0
    input:
        hr
    return: tuple
        (timeOfDay,timeOfDay_encoded
    '''
    if hr in range(0,3) :
        str_val = 'late_night'
        val = 6
    elif hr in range(20,23):
        str_val = 'night'
        val = 5
    elif hr in range(16,20):
        str_val = 'evening'
        val = 4
    elif hr in range(12,26):
        str_val = 'after_noon'
        val = 3
    elif hr in range(8,12):
        str_val = 'morning'
        val = 2
    elif hr in range(3,8):
        str_val = 'early_morning'
        val = 1
    else:
        str_val = 'invalid'
        val = 0
    return (str_val, val)

            # 3-8 early_morning
            # 8-12 morning
            # 12-16 afternoon
            # 16-20 evening
            # 20-00 night
            # 00-3 late_night
            

In [105]:
df['dep_hr'] = df['DepartureDateTime'].dt.hour
df['arr_hr'] = df['ArrivalDateTime'].dt.hour

df['departure_timeOfDay'] = df['dep_hr'].apply(lambda x: time_of_day(x)[1])
df['arrival_timeOfDay'] = df['arr_hr'].apply(lambda x: time_of_day(x)[1])

NameError: name 'time_of_day' is not defined

In [ ]:
X = df.drop(columns=['Price','DepartureDateTime','Duration_timedelta','ArrivalDateTime'])
y = df['Price']

In [ ]:
train(X_transformed,y,regression)
train(X_transformed,y,knearestneighbour)
train(X_transformed,y,decisiontree)
train(X_transformed,y,randomForest)
train(X_transformed,y,xgboost)
train(X_transformed,y,gboost)
train(X_transformed,y,catboost)

## II

In [ ]:
def time_of_day(hr):
    '''
    This function gives the time of day based on logic:
        # 3-8 early_morning or 1
        # 8-12 morning or 2
        # 12-16 afternoon or 3
        # 16-20 evening or 4
        # 20-00 night or 5
        # 00-3 late_night or 6
        # invalid or 0
    input:
        hr
    return: tuple
        (timeOfDay,timeOfDay_encoded
    '''
    if hr in range(0,3) :
        str_val = 'late_night'
        val = 6
    elif hr in range(20,23):
        str_val = 'night'
        val = 5
    elif hr in range(16,20):
        str_val = 'evening'
        val = 4
    elif hr in range(12,26):
        str_val = 'after_noon'
        val = 3
    elif hr in range(8,12):
        str_val = 'morning'
        val = 2
    elif hr in range(3,8):
        str_val = 'early_morning'
        val = 1
    else:
        str_val = 'invalid'
        val = 0
    return (str_val, val)

        # 3-8 early_morning
        # 8-12 morning
        # 12-16 afternoon
        # 16-20 evening
        # 20-00 night
        # 00-3 late_night
        
df['dep_hr'] = df['DepartureDateTime'].dt.hour
df['arr_hr'] = df['ArrivalDateTime'].dt.hour

df['departure_timeOfDay'] = df['dep_hr'].apply(lambda x: time_of_day(x)[0])
df['arrival_timeOfDay'] = df['arr_hr'].apply(lambda x: time_of_day(x)[0])

df['departure_timeOfDay_encoded'] = df['dep_hr'].apply(lambda x: time_of_day(x)[1])
df['arrival_timeOfDay_encoded'] = df['arr_hr'].apply(lambda x: time_of_day(x)[1])

one_hot_cols = ['departure_timeOfDay','arrival_timeOfDay']
df_oneHotEncoded = pd.get_dummies(df[one_hot_cols])

new_df = pd.concat([df,df_oneHotEncoded],axis=1)

new_df.drop(columns=one_hot_cols,inplace=True)

In [ ]:
X = new_df.drop(columns=['Price','DepartureDateTime','Duration_timedelta','ArrivalDateTime'])
y = new_df['Price']

print(X.columns)

train(X_transformed,y,regression)
train(X_transformed,y,knearestneighbour)
train(X_transformed,y,decisiontree)

model = train(X,y,randomForest)
pd.DataFrame({'feature_names_in_':model.feature_names_in_,'feature_importances_':model.feature_importances_}).sort_values(by='feature_importances_',ascending=False)

model = train(X,y,gboost)
pd.DataFrame({'feature_names_in_':model.feature_names_in_,'feature_importances_':model.feature_importances_}).sort_values(by='feature_importances_',ascending=False)

model = train(X,y,xgboost)
importance_gain = model.get_booster().get_score(importance_type='gain')
importance_cover = model.get_booster().get_score(importance_type='cover')
importance_weight = model.get_booster().get_score(importance_type='weight')
importance_total_gain = model.get_booster().get_score(importance_type='total_gain')
importance_total_cover = model.get_booster().get_score(importance_type='total_cover')

pd.DataFrame([importance_weight,importance_gain,importance_total_gain,importance_cover,importance_total_cover],index=['weight','gain','total_gain','cover','total_cover']).T

model = train(X,y,catboost)
pd.DataFrame({'feature_names_in_':X.columns,'feature_importances_':model.get_feature_importance()}).sort_values(by='feature_importances_',ascending=False)

model = train(X,y,randomForest)
pd.DataFrame({'feature_names_in_':model.feature_names_in_,'feature_importances_':model.feature_importances_}).sort_values(by='feature_importances_',ascending=False)

## III

In [106]:
def time_of_day(hr):
    '''
    This function gives the time of day based on logic:
        # 3-8 early_morning or 1
        # 8-12 morning or 2
        # 12-16 afternoon or 3
        # 16-20 evening or 4
        # 20-00 night or 5
        # 00-3 late_night or 6
        # invalid or 0
    input:
        hr
    return: tuple
        (timeOfDay,timeOfDay_encoded
    '''
    if hr in range(0,3) :
        str_val = 'late_night'
        val = 6
    elif hr in range(20,23):
        str_val = 'night'
        val = 5
    elif hr in range(16,20):
        str_val = 'evening'
        val = 4
    elif hr in range(12,26):
        str_val = 'after_noon'
        val = 3
    elif hr in range(8,12):
        str_val = 'morning'
        val = 2
    elif hr in range(3,8):
        str_val = 'early_morning'
        val = 1
    else:
        str_val = 'invalid'
        val = 0
    return (str_val, val)

# 3-8 early_morning
# 8-12 morning
# 12-16 afternoon
# 16-20 evening
# 20-00 night
# 00-3 late_night

df['dep_hr'] = df['DepartureDateTime'].dt.hour
df['arr_hr'] = df['ArrivalDateTime'].dt.hour

df['dep_day_of_week'] = df['DepartureDateTime'].dt.day_of_week 
df['arr_day_of_week'] = df['ArrivalDateTime'].dt.day_of_week 

df['dep_weekday'] = np.where(df["dep_day_of_week"].isin([5,6]),0,1)
df['arr_weekday'] = np.where(df["arr_day_of_week"].isin([5,6]),0,1)


df['departure_timeOfDay'] = df['dep_hr'].apply(lambda x: time_of_day(x)[0])
df['departure_timeOfDay_encoded'] = df['dep_hr'].apply(lambda x: time_of_day(x)[1])
df['arrival_timeOfDay'] = df['arr_hr'].apply(lambda x: time_of_day(x)[0])
df['arrival_timeOfDay_encoded'] = df['arr_hr'].apply(lambda x: time_of_day(x)[1])

one_hot_cols = ['departure_timeOfDay','arrival_timeOfDay']
df_oneHotEncoded = pd.get_dummies(df[one_hot_cols])
new_df = pd.concat([df,df_oneHotEncoded],axis=1)
new_df.drop(columns=one_hot_cols,inplace=True)

In [107]:
X = new_df.drop(columns=['Price','DepartureDateTime','Duration_timedelta','ArrivalDateTime'])
y = new_df['Price']

print(X.columns)

train(X_transformed,y,regression)
train(X_transformed,y,knearestneighbour)
train(X_transformed,y,decisiontree)

Index(['Total_Stops', 'Duration_min', 'Freq_encoded_Airline',
       'Mean_encoded_Airline', 'Freq_encoded_Source',
       'Freq_encoded_Destination', 'Freq_encoded_Route', 'Mean_encoded_Source',
       'Mean_encoded_Destination', 'Mean_encoded_Route',
       'Label_encoded_Airline', 'Label_encoded_Source',
       'Label_encoded_Destination', 'Label_encoded_Route', 'dep_hr', 'arr_hr',
       'dep_day_of_week', 'arr_day_of_week', 'dep_weekday', 'arr_weekday',
       'departure_timeOfDay_encoded', 'arrival_timeOfDay_encoded',
       'departure_timeOfDay_after_noon', 'departure_timeOfDay_early_morning',
       'departure_timeOfDay_evening', 'departure_timeOfDay_late_night',
       'departure_timeOfDay_morning', 'departure_timeOfDay_night',
       'arrival_timeOfDay_after_noon', 'arrival_timeOfDay_early_morning',
       'arrival_timeOfDay_evening', 'arrival_timeOfDay_late_night',
       'arrival_timeOfDay_morning', 'arrival_timeOfDay_night'],
      dtype='object')


ValueError: Input X contains NaN.
LinearRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
X = new_df.drop(columns=['Price','DepartureDateTime','Duration_timedelta','ArrivalDateTime'])
y = new_df['Price']

print(X.columns)

train(X_transformed,y,regression)
train(X_transformed,y,knearestneighbour)
train(X_transformed,y,decisiontree)

model = train(X,y,randomForest)
pd.DataFrame({'feature_names_in_':model.feature_names_in_,'feature_importances_':model.feature_importances_}).sort_values(by='feature_importances_',ascending=False)

model = train(X,y,gboost)
pd.DataFrame({'feature_names_in_':model.feature_names_in_,'feature_importances_':model.feature_importances_}).sort_values(by='feature_importances_',ascending=False)

model = train(X,y,xgboost)
importance_gain = model.get_booster().get_score(importance_type='gain')
importance_cover = model.get_booster().get_score(importance_type='cover')
importance_weight = model.get_booster().get_score(importance_type='weight')
importance_total_gain = model.get_booster().get_score(importance_type='total_gain')
importance_total_cover = model.get_booster().get_score(importance_type='total_cover')

pd.DataFrame([importance_weight,importance_gain,importance_total_gain,importance_cover,importance_total_cover],index=['weight','gain','total_gain','cover','total_cover']).T

model = train(X,y,catboost)
pd.DataFrame({'feature_names_in_':X.columns,'feature_importances_':model.get_feature_importance()}).sort_values(by='feature_importances_',ascending=False)

model = train(X,y,randomForest)
pd.DataFrame({'feature_names_in_':model.feature_names_in_,'feature_importances_':model.feature_importances_}).sort_values(by='feature_importances_',ascending=False)

## IV

In [108]:
def time_of_day(hr):
    '''
    This function gives the time of day based on logic:
        # 3-8 early_morning or 1
        # 8-12 morning or 2
        # 12-16 afternoon or 3
        # 16-20 evening or 4
        # 20-00 night or 5
        # 00-3 late_night or 6
        # invalid or 0
    input:
        hr
    return: tuple
        (timeOfDay,timeOfDay_encoded
    '''
    if hr in range(0,3) :
        str_val = 'late_night'
        val = 6
    elif hr in range(20,23):
        str_val = 'night'
        val = 5
    elif hr in range(16,20):
        str_val = 'evening'
        val = 4
    elif hr in range(12,26):
        str_val = 'after_noon'
        val = 3
    elif hr in range(8,12):
        str_val = 'morning'
        val = 2
    elif hr in range(3,8):
        str_val = 'early_morning'
        val = 1
    else:
        str_val = 'invalid'
        val = 0
    return (str_val, val)
# 3-8 early_morning
# 8-12 morning
# 12-16 afternoon
# 16-20 evening
# 20-00 night
# 00-3 late_night



df['dep_hr'] = df['DepartureDateTime'].dt.hour
df['arr_hr'] = df['ArrivalDateTime'].dt.hour

df['dep_month'] = df['DepartureDateTime'].dt.month
df['dep_day_of_month'] = df['DepartureDateTime'].dt.day

df['arr_month'] = df['ArrivalDateTime'].dt.month
df['arr_day_of_month'] = df['ArrivalDateTime'].dt.day

df['dep_day_of_week'] = df['DepartureDateTime'].dt.day_of_week 
df['arr_day_of_week'] = df['ArrivalDateTime'].dt.day_of_week 

df['dep_weekday'] = np.where(df["dep_day_of_week"].isin([5,6]),0,1)
df['arr_weekday'] = np.where(df["arr_day_of_week"].isin([5,6]),0,1)


df['departure_timeOfDay'] = df['dep_hr'].apply(lambda x: time_of_day(x)[0])
df['departure_timeOfDay_encoded'] = df['dep_hr'].apply(lambda x: time_of_day(x)[1])
df['arrival_timeOfDay'] = df['arr_hr'].apply(lambda x: time_of_day(x)[0])
df['arrival_timeOfDay_encoded'] = df['arr_hr'].apply(lambda x: time_of_day(x)[1])

one_hot_cols = ['departure_timeOfDay','arrival_timeOfDay']
df_oneHotEncoded = pd.get_dummies(df[one_hot_cols])
new_df = pd.concat([df,df_oneHotEncoded],axis=1)
new_df.drop(columns=one_hot_cols,inplace=True)

In [109]:
X = new_df.drop(columns=['Price','DepartureDateTime','Duration_timedelta','ArrivalDateTime'])
y = new_df['Price']

print(X.columns)

train(X_transformed,y,regression)
train(X_transformed,y,knearestneighbour)
train(X_transformed,y,decisiontree)


model = train(X,y,randomForest)
pd.DataFrame({'feature_names_in_':model.feature_names_in_,'feature_importances_':model.feature_importances_}).sort_values(by='feature_importances_',ascending=False)

model = train(X,y,gboost)
pd.DataFrame({'feature_names_in_':model.feature_names_in_,'feature_importances_':model.feature_importances_}).sort_values(by='feature_importances_',ascending=False)

model = train(X,y,xgboost)
importance_gain = model.get_booster().get_score(importance_type='gain')
importance_cover = model.get_booster().get_score(importance_type='cover')
importance_weight = model.get_booster().get_score(importance_type='weight')
importance_total_gain = model.get_booster().get_score(importance_type='total_gain')
importance_total_cover = model.get_booster().get_score(importance_type='total_cover')

pd.DataFrame([importance_weight,importance_gain,importance_total_gain,importance_cover,importance_total_cover],index=['weight','gain','total_gain','cover','total_cover']).T

model = train(X,y,catboost)
pd.DataFrame({'feature_names_in_':X.columns,'feature_importances_':model.get_feature_importance()}).sort_values(by='feature_importances_',ascending=False)

model = train(X,y,randomForest)
pd.DataFrame({'feature_names_in_':model.feature_names_in_,'feature_importances_':model.feature_importances_}).sort_values(by='feature_importances_',ascending=False)

Index(['Total_Stops', 'Duration_min', 'Freq_encoded_Airline',
       'Mean_encoded_Airline', 'Freq_encoded_Source',
       'Freq_encoded_Destination', 'Freq_encoded_Route', 'Mean_encoded_Source',
       'Mean_encoded_Destination', 'Mean_encoded_Route',
       'Label_encoded_Airline', 'Label_encoded_Source',
       'Label_encoded_Destination', 'Label_encoded_Route', 'dep_hr', 'arr_hr',
       'dep_day_of_week', 'arr_day_of_week', 'dep_weekday', 'arr_weekday',
       'departure_timeOfDay_encoded', 'arrival_timeOfDay_encoded', 'dep_month',
       'dep_day_of_month', 'arr_month', 'arr_day_of_month',
       'departure_timeOfDay_after_noon', 'departure_timeOfDay_early_morning',
       'departure_timeOfDay_evening', 'departure_timeOfDay_late_night',
       'departure_timeOfDay_morning', 'departure_timeOfDay_night',
       'arrival_timeOfDay_after_noon', 'arrival_timeOfDay_early_morning',
       'arrival_timeOfDay_evening', 'arrival_timeOfDay_late_night',
       'arrival_timeOfDay_morning', 'a

ValueError: Input X contains NaN.
LinearRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
X = new_df.drop(columns=['Price','DepartureDateTime','Duration_timedelta','ArrivalDateTime'])
y = new_df['Price']

print(X.columns)

train(X_transformed,y,regression)
train(X_transformed,y,knearestneighbour)
train(X_transformed,y,decisiontree)

In [110]:
model = train(X,y,randomForest)
pd.DataFrame({'feature_names_in_':model.feature_names_in_,'feature_importances_':model.feature_importances_}).sort_values(by='feature_importances_',ascending=False)

ValueError: Input X contains NaN.
RandomForestRegressor does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
model = train(X,y,gboost)
pd.DataFrame({'feature_names_in_':model.feature_names_in_,'feature_importances_':model.feature_importances_}).sort_values(by='feature_importances_',ascending=False)

In [ ]:
model = train(X,y,xgboost)
importance_gain = model.get_booster().get_score(importance_type='gain')
importance_cover = model.get_booster().get_score(importance_type='cover')
importance_weight = model.get_booster().get_score(importance_type='weight')
importance_total_gain = model.get_booster().get_score(importance_type='total_gain')
importance_total_cover = model.get_booster().get_score(importance_type='total_cover')

pd.DataFrame([importance_weight,importance_gain,importance_total_gain,importance_cover,importance_total_cover],index=['weight','gain','total_gain','cover','total_cover']).T

In [111]:
model = train(X,y,catboost)
pd.DataFrame({'feature_names_in_':X.columns,'feature_importances_':model.get_feature_importance()}).sort_values(by='feature_importances_',ascending=False)

0.9212487478368616 0.8573681986959675


,feature_names_in_,feature_importances_
9,Mean_encoded_Route,28.144868
23,dep_day_of_month,9.272157
3,Mean_encoded_Airline,8.210206
24,arr_month,7.588095
22,dep_month,7.221982
25,arr_day_of_month,6.846200
1,Duration_min,5.801596
2,Freq_encoded_Airline,3.987141
6,Freq_encoded_Route,3.362545
13,Label_encoded_Route,2.674448


In [112]:
## After doing Time-based feature Engineering we found best model to be xgboost.

In [113]:
model = train(X,y,xgboost)
importance_gain = model.get_booster().get_score(importance_type='gain')
importance_cover = model.get_booster().get_score(importance_type='cover')
importance_weight = model.get_booster().get_score(importance_type='weight')
importance_total_gain = model.get_booster().get_score(importance_type='total_gain')
importance_total_cover = model.get_booster().get_score(importance_type='total_cover')
​
pd.DataFrame([importance_weight,importance_gain,importance_total_gain,importance_cover,importance_total_cover],index=['weight','gain','total_gain','cover','total_cover']).T

SyntaxError: invalid non-printable character U+200B (291284002.py, line 7)

In [ ]:
df_vif= sklearn_vif(exogs=X.columns, data=X).sort_values(by='VIF',ascending=False)

In [ ]:
df_vif

In [ ]:
vif_cols = ['Total_Stops', 'Duration_min', 'Freq_encoded_Airline',
       'Mean_encoded_Airline', 
#             'Freq_encoded_Source',
       'Freq_encoded_Destination', 'Freq_encoded_Route', 
#             'Mean_encoded_Source',
#        'Mean_encoded_Destination',
            'Mean_encoded_Route',
       'Label_encoded_Airline', 'Label_encoded_Source',
       'Label_encoded_Destination', 'Label_encoded_Route', 'dep_hr', 'arr_hr',
#        'departure_timeOfDay_encoded',
            'arrival_timeOfDay_encoded',
       'dep_day_of_week', 'arr_day_of_week', 'dep_weekday', 'arr_weekday',
       'dep_month', 'dep_day_of_month', 
#             'arr_month', 
#             'arr_day_of_month',
       'departure_timeOfDay_after_noon', 
#             'departure_timeOfDay_early_morning',
#        'departure_timeOfDay_evening',
            'departure_timeOfDay_late_night',
       'departure_timeOfDay_morning', 'departure_timeOfDay_night',
       'arrival_timeOfDay_after_noon', 'arrival_timeOfDay_early_morning',
       'arrival_timeOfDay_evening', 
#             'arrival_timeOfDay_late_night',
#        'arrival_timeOfDay_morning', 
#             'arrival_timeOfDay_night'
           ]

X =X[vif_cols]

df_vif= sklearn_vif(exogs=X.columns, data=X).sort_values(by='VIF',ascending=False)
df_vif

In [ ]:
model = train(X[vif_cols],y,xgboost)

importance_gain = model.get_booster().get_score(importance_type='gain')
importance_cover = model.get_booster().get_score(importance_type='cover')
importance_weight = model.get_booster().get_score(importance_type='weight')
importance_total_gain = model.get_booster().get_score(importance_type='total_gain')
importance_total_cover = model.get_booster().get_score(importance_type='total_cover')

pd.DataFrame([importance_weight,importance_gain,importance_total_gain,importance_cover,importance_total_cover],index=['weight','gain','total_gain','cover','total_cover']).T

## Cross Validation

In [ ]:
from sklearn.model_selection import cross_validate

In [ ]:
from xgboost import XGBRegressor
Xgboost_reg = XGBRegressor()

In [ ]:
estimator = Xgboost_reg
X = X[vif_cols]
y = y
number_of_cross_validation = 5

def basic_cross_validation(estimator,X,y,number_of_cross_validation=5):
    scores = cross_validate(estimator, X, y, cv=number_of_cross_validation,
                             scoring=('r2', 'neg_mean_squared_error'),
                             return_train_score=True)

    print(scores['train_r2'])
    print(scores['train_r2'].mean())

    print(scores['test_r2'])
    print(scores['test_r2'].mean())

In [ ]:
from catboost import CatBoostRegressor

In [ ]:
basic_cross_validation(CatBoostRegressor(verbose=False),X[vif_cols],y,number_of_cross_validation=5)

#### Actions taken in feature Engineering

In [ ]:
def time_of_day(hr):
    '''
    This function gives the time of day based on logic:
        # 3-8 early_morning or 1
        # 8-12 morning or 2
        # 12-16 afternoon or 3
        # 16-20 evening or 4
        # 20-00 night or 5
        # 00-3 late_night or 6
        # invalid or 0
    input:
        hr
    return: tuple
        (timeOfDay,timeOfDay_encoded
    '''
    if hr in range(0,3) :
        str_val = 'late_night'
        val = 6
    elif hr in range(20,23):
        str_val = 'night'
        val = 5
    elif hr in range(16,20):
        str_val = 'evening'
        val = 4
    elif hr in range(12,26):
        str_val = 'after_noon'
        val = 3
    elif hr in range(8,12):
        str_val = 'morning'
        val = 2
    elif hr in range(3,8):
        str_val = 'early_morning'
        val = 1
    else:
        str_val = 'invalid'
        val = 0
    return (str_val, val)


def time_based_feature_Engineering(df):

    df['dep_hr'] = df['DepartureDateTime'].dt.hour
    df['arr_hr'] = df['ArrivalDateTime'].dt.hour

    df['dep_month'] = df['DepartureDateTime'].dt.month
    df['dep_day_of_month'] = df['DepartureDateTime'].dt.day

    df['arr_month'] = df['ArrivalDateTime'].dt.month
    df['arr_day_of_month'] = df['ArrivalDateTime'].dt.day

    df['dep_day_of_week'] = df['DepartureDateTime'].dt.day_of_week 
    df['arr_day_of_week'] = df['ArrivalDateTime'].dt.day_of_week 

    df['dep_weekday'] = np.where(df["dep_day_of_week"].isin([5,6]),0,1)
    df['arr_weekday'] = np.where(df["arr_day_of_week"].isin([5,6]),0,1)


    df['departure_timeOfDay'] = df['dep_hr'].apply(lambda x: time_of_day(x)[0])
    df['departure_timeOfDay_encoded'] = df['dep_hr'].apply(lambda x: time_of_day(x)[1])
    df['arrival_timeOfDay'] = df['arr_hr'].apply(lambda x: time_of_day(x)[0])
    df['arrival_timeOfDay_encoded'] = df['arr_hr'].apply(lambda x: time_of_day(x)[1])

    one_hot_cols = ['departure_timeOfDay','arrival_timeOfDay']
    df_oneHotEncoded = pd.get_dummies(df[one_hot_cols])
    new_df = pd.concat([df,df_oneHotEncoded],axis=1)
    
    drop_cols = [
                'DepartureDateTime',
                'Duration_timedelta',
                'ArrivalDateTime',
                'Freq_encoded_Source',
                'Mean_encoded_Source',
                'Mean_encoded_Destination',
                'departure_timeOfDay_encoded',
                'arr_month', 
                'arr_day_of_month',
                'departure_timeOfDay_early_morning',
                'departure_timeOfDay_evening',
                'arrival_timeOfDay_late_night',
                'arrival_timeOfDay_morning', 
                'arrival_timeOfDay_night']
    
    new_df.drop(columns=one_hot_cols,inplace=True)
    new_df.drop(columns=drop_cols,inplace=True)
    return new_df

In [ ]:
final_df = time_based_feature_Engineering(df)

In [ ]:
final_df.to_csv("FinalData.csv",index=False)

In [ ]:
final_df.columns

In [ ]:
X = final_df.drop(columns='Price')
y = final_df['Price']
model = train(X,y,xgboost)

importance_gain = model.get_booster().get_score(importance_type='gain')
importance_cover = model.get_booster().get_score(importance_type='cover')
importance_weight = model.get_booster().get_score(importance_type='weight')
importance_total_gain = model.get_booster().get_score(importance_type='total_gain')
importance_total_cover = model.get_booster().get_score(importance_type='total_cover')

pd.DataFrame([importance_weight,importance_gain,importance_total_gain,importance_cover,importance_total_cover],index=['weight','gain','total_gain','cover','total_cover']).T

In [ ]:
model

## Paramter optimization

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

In [ ]:
param_grid = {
    "learning_rate" : [0.1,0.2,0.3,0.5,],
    "max_depth" : [4,5,6,7],
    "n_estimators" : [100,200,300,400,500]
}

In [ ]:
reg = RandomizedSearchCV(XGBRegressor(),param_grid,random_state=0)

In [ ]:
model = reg.fit(X,y)

In [ ]:
model.best_estimator_

In [ ]:
model

In [ ]:
basic_cross_validation(model.best_estimator_,X,y,number_of_cross_validation=5)

In [114]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
xgboost_regressor = model.best_estimator_
xgboost_regressor.fit(X_train, y_train)
print(metrics.r2_score(y_train,xgboost_regressor.predict(X_train)),metrics.r2_score(y_test,xgboost_regressor.predict(X_test)))

AttributeError: 'CatBoostRegressor' object has no attribute 'best_estimator_'

In [115]:
param_grid = {
    "learning_rate" : [0.2,0.21,0.29],
    "n_estimators" : [95,100,105]
}

In [116]:
reg = GridSearchCV(XGBRegressor(),param_grid)

In [117]:
model = reg.fit(X,y)

ValueError: 
All the 45 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
45 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\user\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\user\anaconda3\Lib\site-packages\xgboost\core.py", line 506, in inner_f
    return f(**kwargs)
           ^^^^^^^^^^^
  File "C:\Users\user\anaconda3\Lib\site-packages\xgboost\sklearn.py", line 761, in fit
    train_dmatrix, evals = _wrap_evaluation_matrices(
                           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\user\anaconda3\Lib\site-packages\xgboost\sklearn.py", line 286, in _wrap_evaluation_matrices
    train_dmatrix = create_dmatrix(
 

In [ ]:
model.best_estimator_

In [ ]:
model

In [ ]:
basic_cross_validation(model.best_estimator_,X,y,number_of_cross_validation=5)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
xgboost_regressor = model.best_estimator_
xgboost_regressor.fit(X_train, y_train)
print(metrics.r2_score(y_train,xgboost_regressor.predict(X_train)),metrics.r2_score(y_test,xgboost_regressor.predict(X_test)))

In [ ]:
# XGBRegressor( learning_rate=0.21, max_depth=6,  n_estimators=100)

In [ ]:
import pickle

In [118]:
with open('../models/xgboost.pickle','wb') as file:
        pickle.dump(xgboost_regressor,file)

FileNotFoundError: [Errno 2] No such file or directory: '../models/xgboost.pickle'